In [1]:
#Part B

import numpy as np


inputs = np.zeros((16,5)) 

# For generating the all possible input values, concatenated with the bias vector. 5th column is all 1, which
# represents the bias vector.
def generateInputVector(inputVector):
    for i in range(np.shape(inputVector)[0]):        
        temp = bin(i).split('b')
        temp = temp[-1]
        rowVector = [int(x) for x in str(temp)] 
        rowVector.append(1)
        sizeRowVector = len(rowVector)
        inputVector[i, (np.shape(inputVector)[1]) - sizeRowVector:] = rowVector[:]
    return inputVector

inputs = generateInputVector(inputs)
print(inputs)

[[0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 1.]
 [0. 0. 1. 0. 1.]
 [0. 0. 1. 1. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 1. 1.]
 [0. 1. 1. 0. 1.]
 [0. 1. 1. 1. 1.]
 [1. 0. 0. 0. 1.]
 [1. 0. 0. 1. 1.]
 [1. 0. 1. 0. 1.]
 [1. 0. 1. 1. 1.]
 [1. 1. 0. 0. 1.]
 [1. 1. 0. 1. 1.]
 [1. 1. 1. 0. 1.]
 [1. 1. 1. 1. 1.]]


In [2]:
# The activation function which the the unit step function.
def unitStepFunction(k):
    k = np.where(k<0, 0,1)
    return k

In [3]:
# Hand calculated weights for hidden layer and output layer.

hiddenweights = np.array([[5, 0, 5, 5, -13],
                           [0, -4, 5, 5, -9],
                           [-6, 11, -5, 0, -10],
                           [-5, 13, 0, -6, -11]])

outputweights = np.array([2, 3, 5, 7, -1])

In [4]:
# Testing the network with given weights.

hiddenOutsTemp = unitStepFunction(inputs.dot(hiddenweights.T))

# Again, adding the bias vector before going into output neuron.
bias = np.ones((16,1))
hiddenOuts = np.concatenate((hiddenOutsTemp, bias),axis =1)
print(hiddenOuts)

[[0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 1.]
 [0. 0. 1. 1. 1.]
 [0. 0. 1. 0. 1.]
 [0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [1. 1. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 1.]]


In [5]:
# Final output for the network with given weights.

outputs = unitStepFunction(hiddenOuts.dot(outputweights.T))
print(outputs)

[0 0 0 1 1 1 1 0 0 0 0 1 0 0 0 1]


In [6]:
# Implementation of the logic function. Therefore, we can test the performance of the network above.

def xor_gate(a, b):
    result = (a and (not b)) or ((not a) and b)
    return result

def logicFunction(inputVector):
    result = list()
    for i in range(np.shape(inputVector)[0]):
        temp = xor_gate(inputVector[i,0] or (not inputVector[i,1]), (not inputVector[i,2]) or (not inputVector[i,3]))
        result.append(temp)
    return result

In [7]:
# The outputs from the logic function. We will check this output with the ones we have found above.
outputsCheck = logicFunction(inputs[:,0:4])
print(outputsCheck)

[False, False, False, True, True, True, True, False, False, False, False, True, False, False, False, True]


In [8]:
# For calculating the accuracy of the network, accuracyCalc function is defined.

def accuracyCalc(x, y):
    result = 0
    count = 0
    size = np.shape(x)[0]
    sentence = 'The accuracy of the model is: '
    for i in range(size):
        if (x[i] == y[i]):
            count = count +1
    result = (count / size) * 100
    
    return result

In [9]:
# The accuracy result between the network and the logic function itself.

accuracy = accuracyCalc(outputs, outputsCheck)
print('The accuracy of the model is: ' + str(accuracy))

The accuracy of the model is: 100.0


In [10]:
# Part C

# Robust Weights
# w_rh = robust weights for hidden layer
# w_ro = robust weights for output layer

w_rh = np.array([[1, 0, 1, 1, -2.5],
                           [0, -1, 1, 1, -1.5],
                           [-1, 1, -1, 0, -0.5],
                           [-1, 1, 0, -1, -0.5]])

w_ro = np.array([1, 1, 1, 1, -0.5])

In [11]:
# Part D

# Generate 400 input samples by concatenating 25 samples from each input.
inputsWithNoise = np.zeros((400,5))
for k in range(25):
    for i in range(np.shape(inputs)[0]):
        inputsWithNoise[(k*16)+i,:] = inputs[i,:]

# Then check the outputs of the inputsWithNoise.
outputsCheck_D = logicFunction(inputsWithNoise[:,0:4])

# Create a gaussian noise with 0 mean and 0.2 std. Then add this noise to the inputsWithNoise array.
# np.random.seed(7) is for getting the same output each run.
np.random.seed(7)
gaussianNoise = np.random.normal(loc=0, scale=0.2, size=1600).reshape(400, 4)
inputsWithNoise[:, 0:4] += gaussianNoise        

In [12]:
# Then test the inputsWithNoise with the random weights network.

outputTemp = unitStepFunction(inputsWithNoise.dot(hiddenweights.T))
bias = np.ones((400,1))
hiddenOutsTemp = np.concatenate((outputTemp, bias),axis =1)

random_network_output = unitStepFunction(hiddenOutsTemp.dot(outputweights.T))

In [13]:
# Then test the inputsWithNoise with the robust weights network.

robustOutputTemp = unitStepFunction(inputsWithNoise.dot(w_rh.T))
bias = np.ones((400,1))
robustHiddenOuts = np.concatenate((robustOutputTemp, bias),axis =1)

robust_network_output = unitStepFunction(robustHiddenOuts.dot(w_ro.T))

In [14]:
# Accuracy of the random weight network.

accuracy = accuracyCalc(random_network_output, outputsCheck_D)
print('The accuracy of the random weighted network is: ' + str(accuracy))

The accuracy of the random weighted network is: 86.75


In [15]:
# Accuracy of the robust network.

accuracy = accuracyCalc(robust_network_output, outputsCheck_D)
print('The accuracy of the robust network is: ' + str(accuracy))

The accuracy of the robust network is: 90.0
